In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Step 1: Load your sentiment data
def load_sentiment_data(file_path):
    # Assumes CSV with 'time' and 'sentiment' columns
    data = pd.read_csv(file_path)
    # Ensure sentiment is between -1 and 1
    data['sentiment_score'] = data['sentiment_score'].clip(-1, 1)
    return data

# Step 2: Define metrics
def MSE(y_true, y_pred):
    return mean_squared_error(y_true, y_pred)

def RMSE(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def MAE(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def sRMSE(y_true, y_pred, scale):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return rmse / scale

def MASE(y_true, y_pred, y_naive):
    errors = np.abs(y_true - y_pred)
    naive_errors = np.abs(y_true - y_naive)
    return np.mean(errors) / np.mean(naive_errors[naive_errors != 0] + 1e-10)

def sPIS(y_true, y_pred, scale):
    errors = y_pred - y_true
    return np.mean(errors) / scale  # Mean bias scaled

def sAPIS(y_true, y_pred, scale):
    errors = y_pred - y_true
    return (np.mean(np.abs(errors)) + np.abs(np.mean(errors))) / scale  # Bias + variance

def SMAPE(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred) + 1e-10))

# Step 3: Test reliability
def test_metrics(file_path, n_samples=None):
    # Load data
    data = load_sentiment_data(file_path)
    y_true = data['sentiment_score'].values
    if n_samples:
        y_true = y_true[:n_samples]  # Limit data size for testing
    scale = np.std(y_true) if np.std(y_true) > 0 else 1
    # Naive forecast: previous value
    y_naive = np.roll(y_true, 1)
    y_naive[0] = y_true[0]  # Handle first value

    # Define noise parameters
    bias_values = [-0.2, 0, 0.2]  # Small bias for -1 to 1 range
    variance_values = [0.01, 0.1, 0.5]  # Small variance for sentiment
    results = []

    # Add noise and compute metrics
    for bias in bias_values:
        for var in variance_values:
            # Add noise and clip to [-1, 1]
            noise = np.random.normal(bias, np.sqrt(var), len(y_true))
            y_pred = np.clip(y_true + noise, -1, 1)

            # Compute metrics
            metrics = {
                "bias": bias,
                "var": var,
                "MSE": MSE(y_true, y_pred),
                "RMSE": RMSE(y_true, y_pred),
                "MAE": MAE(y_true, y_pred),
                "sRMSE": sRMSE(y_true, y_pred, scale),
                "MASE": MASE(y_true, y_pred, y_naive),
                "sPIS": sPIS(y_true, y_pred, scale),
                "sAPIS": sAPIS(y_true, y_pred, scale),
                "SMAPE": SMAPE(y_true, y_pred)
            }
            results.append(metrics)

    # Step 4: Analyze reliability
    results_df = pd.DataFrame(results)
    reliability = {}
    for metric in ["MSE", "RMSE", "MAE", "sRMSE", "MASE", "sPIS", "sAPIS", "SMAPE"]:
        # Variability (standard deviation)
        variability = results_df[metric].std()

        # Bootstrap confidence intervals
        boot_values = []
        for _ in range(100):
            sample = results_df[metric].sample(frac=1, replace=True)
            boot_values.append(sample.mean())
        ci = np.percentile(boot_values, [2.5, 97.5])
        ci_width = ci[1] - ci[0]

        reliability[metric] = {"variability": variability, "ci_width": ci_width}

    # Step 5: Print results
    print("Metric Reliability (Lower variability and CI width = more reliable):")
    for metric, stats in reliability.items():
        print(f"{metric}: Variability = {stats['variability']:.4f}, CI Width = {stats['ci_width']:.4f}")

    # Plot sensitivity to bias
    for metric in ["MSE", "RMSE", "MAE", "sRMSE", "MASE", "sPIS", "sAPIS", "SMAPE"]:
        plt.figure(figsize=(8, 6))
        plt.scatter(results_df["bias"], results_df[metric], alpha=0.5)
        plt.title(f"{metric} Sensitivity to Bias")
        plt.xlabel("Bias")
        plt.ylabel(metric)
        plt.savefig(f"{metric}_sensitivity.png")
        plt.close()

    return results_df, reliability

# Run the test
# Replace 'your_data.csv' with the path to your CSV file
results_df, reliability = test_metrics('/content/Score_output_data.csv', n_samples=1000)

Metric Reliability (Lower variability and CI width = more reliable):
MSE: Variability = 0.1296, CI Width = 0.1544
RMSE: Variability = 0.1747, CI Width = 0.2173
MAE: Variability = 0.1333, CI Width = 0.1537
sRMSE: Variability = 0.3247, CI Width = 0.4015
MASE: Variability = 0.3112, CI Width = 0.3709
sPIS: Variability = 0.2676, CI Width = 0.3380
sAPIS: Variability = 0.3014, CI Width = 0.3389
SMAPE: Variability = 23.1660, CI Width = 26.1985
